### INFO 180 – Metodar i kunstig intelligens
### Oblig-oppgåve 4 – Maskinlæring

### Av Hans Martin Aannestad

In [1]:
import pandas as pd
import scipy as sc
import numpy as np

# Load libraries
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


In [2]:
names = ['Survived','Pclass','Name','Sex','Age','Siblings/Spouses Aboard','Parents/Children Aboard','Fare']
df = read_csv('titanic.csv', names=names)
df = df.drop('Name',1)   # Fjerner navn
#df.assign(Sex=pd.factorize(df['Sex'])[0])
df['Sex'] = df['Sex'].replace(['male','female'],[0,1])     # gjør om til binært
df

C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/450378945.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Name',1)   # Fjerner navn


,Survived,Pclass,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250
3,1,1,1,35.0,1,0,53.1000
4,0,3,0,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
882,0,2,0,27.0,0,0,13.0000
883,1,1,1,19.0,0,0,30.0000
884,0,3,1,7.0,1,2,23.4500
885,1,1,0,26.0,0,0,30.0000


In [3]:
scaler = MinMaxScaler()
# Splitter før man modifiserer datasettet
df_train, df_test = train_test_split(df, test_size=0.2,random_state=1)

# Skalerer verider
df_train = pd.DataFrame(scaler.fit_transform(df_train.values), columns=df_train.columns, index=df_train.index)
df_test = pd.DataFrame(scaler.fit_transform(df_test.values), columns=df_test.columns, index=df_test.index)

y_train = df_train['Survived']
X_train = df_train.drop('Survived',1)

y_test = df_test['Survived']
X_test = df_test.drop('Survived',1)

C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/1416223592.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = df_train.drop('Survived',1)
C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/1416223592.py:13: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_test = df_test.drop('Survived',1)


• K-næraste nabo: her må de gå gjennom heile datasettet og erstatte alle verdiar for Sex
(male, female) med 0 eller 1. Det er og enklast om de fjernar namnekolonna. I tillegg bør de
sørge for ei skalering av verdiane i kvar kolonne, slik at dei vert konvertert til intervallet [0,1].
Dette gjer ein fordi dei ulike kolonnene har ulike verdispenn, og dei med stort verdispenn vil 
telje mykje meir enn dei med lite verdispenn om ein ikkje skalerer. Skalering kan de gjere
med verktøyet minMaxScaler som fins i programpakken sklearn. De ser korleis den kan
brukast her:
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
De skal bruke sklearn sin K-nearest neighbour-klasse:
https://scikitlearn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
De treng ikkje tenke på anna enn val av k, dvs. tal på naboar som skal brukast i klassifiseringa
som parameter her. Om de ikkje veljer k, vert den sett til 5.

In [4]:
# K NEAREST NEIGHBORS (KNN), auto pipeline:

# Optimering av hyperparameter K på treningsdata

#leaf_size = list(range(1,30))
#n_neighbors = list(range(1,20))
#p=[1,2]

#hyperparameters = dict(leaf_size=list(range(1,30)), n_neighbors=list(range(1,25)), p=[1,2])
#clf = GridSearchCV(KNN(), hyperparameters, cv=10)
#opt_knn = clf.fit(X_train,y_train)

#print('leaf_size:', opt_knn.best_estimator_.get_params()['leaf_size'])
#print('p:', opt_knn.best_estimator_.get_params()['p'])
#print('n_neighbors:', opt_knn.best_estimator_.get_params()['n_neighbors'])

# Velg beste modell, evaluer på test data
#y_pred = opt_knn.predict(X_test)
#print (f'KNN accuracy: {accuracy_score(y_test,y_pred):.3f}')

In [10]:
# Gjorde det til slutt enkelt: kjør enkel modell på test data

opt_knn = KNN(n_neighbors=6)
opt_knn = opt_knn.fit(X_train, y_train)

y_train_pred = opt_knn.predict(X_train)
y_test_pred = opt_knn.predict(X_test)

print (f'KNN train accuracy: {accuracy_score(y_train,y_train_pred):.3f}')
print (f'KNN test accuracy: {accuracy_score(y_test,y_test_pred):.3f}')

KNN train accuracy: 0.843
KNN test accuracy: 0.831


• Naiv bayes: her må de gå gjennom heile datasettet og konvertere dei numeriske verdiane i
Age og Fare til kvalitative verdiar (for eksempel ‘ung’, ’ganske ung’, ‘ganske gammal’,
‘gammal’). Dette gjer de ved å finne kvartilane (25%, 50%, og 75%-persentilane) for dei to
kolonnene og bruke desse grensene til velje gruppe for kvar passasjer. Dette gjer ein fordi
såkalla «kategorisk naiv bayes» brukar kategoriar i læringsalgoritmen og ikkje tal. De kan
bruke biblioteket numpy til å finne grenser for kvartilane:
https://numpy.org/doc/stable/reference/generated/numpy.percentile.html
Etterpå må de lage kolonner som inneheld rett kvartil for kvar passasjer.
De skal bruke sklearn sin kategoriske naiv bayes:
https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.CategoricalNB.html
De treng ikkje tenke på parametrar her.

In [6]:

from sklearn.naive_bayes import CategoricalNB

# (25%, 50%, og 75%-persentilane)

# Percentile: 1-25,25-50,50-75,75-100
# Bin:          1    2     3      4

age25 = np.percentile(df['Age'],25)
age50 = np.percentile(df['Age'],50)
age75 = np.percentile(df['Age'],75)

# Age -> categorical
df['Age'] = df['Age'].astype(int)
df.loc[df['Age'] <= age25, 'Age'] = 1
df.loc[(df['Age'] > age25) & (df['Age'] <= age50), 'Age'] = 2
df.loc[(df['Age'] > age50) & (df['Age'] <= age75), 'Age'] = 3
df.loc[df['Age'] > age75, 'Age'] = 4

fare25 = np.percentile(df['Fare'],25)
fare50 = np.percentile(df['Fare'],50)
fare75 = np.percentile(df['Fare'],75)

# Fare -> categorical
df['Fare'] = df['Fare'].astype(int)
df.loc[df['Fare'] <= fare25, 'Fare'] = 1
df.loc[(df['Fare'] > fare25) & (df['Fare'] <= fare50), 'Fare'] = 2
df.loc[(df['Fare'] > fare50) & (df['Fare'] <= fare75), 'Fare'] = 3
df.loc[df['Fare'] > fare75, 'Fare'] = 4


In [12]:
from sklearn.naive_bayes import GaussianNB  # could not get CategoricalNB to work for prediction so used GaussianNB

df_train, df_test = train_test_split(df, test_size=0.2,random_state=1)

y_train = df_train['Survived']
X_train = df_train.drop('Survived',1)

y_test = df_test['Survived']
X_test = df_test.drop('Survived',1)

clf = GaussianNB()
#clf = CategoricalNB()
clf.fit(X_train, y_train)

y_train_pred=clf.predict(X_train)
y_test_pred=clf.predict(X_test)
print (f'Naive Bayes train accuracy: {accuracy_score(y_train,y_train_pred):.3f}')
print (f'Naive Bayes test accuracy: {accuracy_score(y_test,y_test_pred):.3f}')

Naive Bayes train accuracy: 0.793
Naive Bayes test accuracy: 0.775


C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/2384148492.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = df_train.drop('Survived',1)
C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/2384148492.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_test = df_test.drop('Survived',1)


• Logistisk regresjon: Denne metoden er ikkje forelest, men de kan finne informasjon her:
https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
Her kan de bruke same datasettet som for k-næraste nabo, både den skalerte og ikkjeskalerte versjonen går nok bra.

In [13]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

# Her kan de bruke same datasettet som for k-næraste nabo:

names = ['Survived','Pclass','Name','Sex','Age','Siblings/Spouses Aboard','Parents/Children Aboard','Fare']
df = read_csv('titanic.csv', names=names)
df = df.drop('Name',1)   # Fjerner navn
df['Sex'] = df['Sex'].replace(['male','female'],[0,1])   #df.assign(Sex=pd.factorize(df['Sex'])[0])

scaler = MinMaxScaler()
# Splitter før man modifiserer datasettet
df_train, df_test = train_test_split(df, test_size=0.2,random_state=1)

# Skalerer verider
df_train = pd.DataFrame(scaler.fit_transform(df_train.values), columns=df_train.columns, index=df_train.index)
df_test = pd.DataFrame(scaler.fit_transform(df_test.values), columns=df_test.columns, index=df_test.index)

y_train = df_train['Survived']
X_train = df_train.drop('Survived',1)

y_test = df_test['Survived']
X_test = df_test.drop('Survived',1)

# Logistisk regresjon:

logreg.fit(X_train,y_train)

y_train_pred=logreg.predict(X_train)
y_test_pred=logreg.predict(X_test)

# import the metrics class
from sklearn import metrics
cnf_matrix = metrics.confusion_matrix(y_test, y_test_pred)
cnf_matrix

print (f'Logistisk regresjon train accuracy: {accuracy_score(y_train,y_train_pred):.3f}')
print (f'Logistisk regresjon test accuracy: {accuracy_score(y_test,y_test_pred):.3f}')



Logistisk regresjon train accuracy: 0.800
Logistisk regresjon test accuracy: 0.803


C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/556006969.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df = df.drop('Name',1)   # Fjerner navn
C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/556006969.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = df_train.drop('Survived',1)
C:\Users\hmaan\AppData\Local\Temp/ipykernel_13212/556006969.py:23: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_test = df_test.drop('Survived',1)
